## Import

In [69]:
import pandas as pd
import joblib

## Load Data

In [92]:
#Carregando e ajustando o dataset
df = pd.read_csv("date_state_covidbr.csv",sep=";")
df.date = pd.to_datetime(df.date)
df.columns

Index(['date', 'state', 'cases', 'deaths', 'Latitude', 'Longitude',
       'Densidade', 'area_state', 'Região', 'UF', 'Population_2019', 'Day',
       'new_cases', 'new_deaths', 'Semana_do_Ano', 'Dia_do_Ano', 'Mes', 'Dia',
       'Isolamento', 'Dias_de_Isolamento', 'IIS_BR', 'IIS_per_Cases',
       'count_cities', 'Population_2019_cities', 'area_cities', 'cases_cities',
       'Densidade_cities', 'Hab_per_Case_cities', 'Case_per_Km2_cities',
       'Hab_per_Cases_per_Km2_cities', 'infec_state_area',
       'infec_state_population', 'cases_font2', 'new_cases_font2',
       'Cases_Last_Day', 'Diff_Cases_Last_Day', 'Cases_Last_Day_1',
       'Diff_Cases_Last_Day_1', 'Cases_Last_Day_2', 'Diff_Cases_Last_Day_2',
       'Cases_por_Population_Last_Day', 'Cases_por_Densidade_Last_Day',
       'Mean_Cases_Last_Days_3', 'Desvio_Cases_Last_Days_3',
       'Mean_Cases_Last_Days_7', 'Desvio_Cases_Last_Days_7',
       'Mean_Diff_Cases_Last_Days_3', 'Max_Diff_Cases_Last_Days_3',
       'Desvio_Diff_C

## Load Model

In [93]:
#Carregando o modelo
model = joblib.load("model_RID.pkl")
model

Ridge(alpha=0.03, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=0, solver='auto', tol=0.001)

## Criando "Pipeline" para Forecast

In [94]:
#Features do modelo em ordem de implementação no predict
feat_mod = ['Cases_Last_Day', 'UF_CAT_TO', 'UF_CAT_SE', 'UF_CAT_AL', 'UF_CAT_RO',
            'UF_CAT_GO', 'UF_CAT_MS', 'UF_CAT_PI', 'UF_CAT_AC', 'UF_CAT_PR']
#Features necessárias
feat = ['date', 'state', 'UF', 'cases']
#Todas as features
feat.extend(feat_mod)

In [107]:
#Selecionando apenas as features do modelo a partir do dataset
df = df.loc[:,feat]
df.head(5)

,date,state,UF,cases,Cases_Last_Day,UF_CAT_TO,UF_CAT_SE,UF_CAT_AL,UF_CAT_RO,UF_CAT_GO,UF_CAT_MS,UF_CAT_PI,UF_CAT_AC,UF_CAT_PR
0,2020-01-30,Acre,AC,0,NaN,0,0,0,0,0,0,0,1,0
1,2020-01-31,Acre,AC,0,0.0,0,0,0,0,0,0,0,1,0
2,2020-02-01,Acre,AC,0,0.0,0,0,0,0,0,0,0,1,0
3,2020-02-02,Acre,AC,0,0.0,0,0,0,0,0,0,0,1,0
4,2020-02-03,Acre,AC,0,0.0,0,0,0,0,0,0,0,1,0


In [110]:
#Selecionando os estados para previsão
ufs = [ 'AC' ] #, 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT','MS', 'MG', 'PR', 'PB', 'PA', 'PE', 'PI', 'RN', 'RS', 'RJ', 'RO', 'RR', 'SC', 'SE', 'SP', 'TO' ]

#Definindo o número de dias para previsão
dias = 1

for uf in range(0,len(ufs)):
    
    for i in range(0,dias):
        
        #Selecionando estado para previsão
        forecast = df[df['UF']==ufs[uf]]

        #Selecionando features para previsão do modelo
        #Features do modelo em ordem de implementação no predict
        #'Cases_Last_Day', 'UF_CAT_TO', 'UF_CAT_SE', 'UF_CAT_AL', 'UF_CAT_RO',
        #'UF_CAT_GO', 'UF_CAT_MS', 'UF_CAT_PI', 'UF_CAT_AC', 'UF_CAT_PR'
        cld = forecast.Cases_Last_Day.tail(1).values[0]
        to = forecast.UF_CAT_TO.tail(1).values[0]
        se = forecast.UF_CAT_SE.tail(1).values[0]
        al = forecast.UF_CAT_AL.tail(1).values[0]
        ro = forecast.UF_CAT_RO.tail(1).values[0]
        go = forecast.UF_CAT_GO.tail(1).values[0]
        ms = forecast.UF_CAT_MS.tail(1).values[0]
        pi = forecast.UF_CAT_PI.tail(1).values[0]
        ac = forecast.UF_CAT_AC.tail(1).values[0]
        pr = forecast.UF_CAT_PR.tail(1).values[0]

        #Prevendo o número de casos do próximo dia
        pred = model.predict([[cld,to,se,al,ro,go,ms,pi,ac,pr]])[0]
        
        #Criando linha de dados do próximo dia
        att = pd.DataFrame({'date':'NOVO DIA','state':forecast.state[0],'UF':ufs[uf],'cases':pred,
                   'Cases_Last_Day':cld,
                   'UF_CAT_TO':to,'UF_CAT_SE':se,'UF_CAT_AL':al,'UF_CAT_RO':ro,'UF_CAT_GO':go,
                   'UF_CAT_MS':ms,'UF_CAT_PI':pi,'UF_CAT_AC':ac,'UF_CAT_PR':pr},index=[0])

        #Atualizando dataset de previsão
        forecast = pd.concat([forecast,att]).reset_index(drop=True)
        
forecast.tail(5)

,date,state,UF,cases,Cases_Last_Day,UF_CAT_TO,UF_CAT_SE,UF_CAT_AL,UF_CAT_RO,UF_CAT_GO,UF_CAT_MS,UF_CAT_PI,UF_CAT_AC,UF_CAT_PR
73,2020-04-12 00:00:00,Acre,AC,77.000000,72.0,0,0,0,0,0,0,0,1,0
74,2020-04-13 00:00:00,Acre,AC,90.000000,77.0,0,0,0,0,0,0,0,1,0
75,2020-04-14 00:00:00,Acre,AC,99.000000,90.0,0,0,0,0,0,0,0,1,0
76,2020-04-15 00:00:00,Acre,AC,101.000000,99.0,0,0,0,0,0,0,0,1,0
77,NOVO DIA,Acre,AC,104.505112,99.0,0,0,0,0,0,0,0,1,0
